# Transmission assumptions
In the previous chapters, 
we have been using frequency-dependent transmission assumption.
Under this assumption, the rate of transmission is proportional
to the number of susceptibles and the prevalence of infectious persons in the population.
We have been referring to the parameter used for this transition process as the `contact_rate`,
in which case we can consider this as the _per capita_ rate at which
two specific individuals come into effective contact.
Note that an effective contact can be defined as a contact that would result in
transmission were it to occur between a susceptible person and an infectious person.
Let's return to a really simple SIR model.
In fact, even simpler than the one we introduced in notebook 02,
because it doesn't even have an infection-related death flow.
We'll initially create a function that returns the model object without
any transmission flows, so we can add the process we want later.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def get_sir_base_structure(
    config: dict,
) -> CompartmentalModel:
    """
    Generate a mode that doesn't do much in itself, but has basic
    characteristics that we can then use to add our transmission assumptions to.
    
    Args:
        config: The fixed values used in creating the model structure
    Returns:
        The summer model object
    """
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    infectious_compartment = ["infectious"]
    analysis_times = (0.,  config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    pop = config["population"]
    seed = config["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0., msg
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery"), 
        source="infectious", 
        dest="recovered",
    )
    return model

## Frequency dependence
As mentioned, 
we've already been using the assumption of frequency-dependent transmission in previous notebooks, 
but let's have another look at the model using this type of transmission.

In [ ]:
model_config = {
    "population": 1.,
    "seed": 0.01,
    "end_time": 20.,
}
freq_parameters = {
    "recovery": 0.333,
    "contact_rate": 1.,
}

In [ ]:
sir_freq_model = get_sir_base_structure(model_config)
sir_freq_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
sir_freq_model.run(parameters=freq_parameters)
sir_freq_values = sir_freq_model.get_outputs_df()
sir_freq_values.plot(
    labels={"index": "time", "value": "proportion"},
)

For all our previous models,
and as briefly shown in the ordinary differential equations presented in notebook 07,
we have previously assumed that the force of infection scales according
to the prevalence of infectious individuals in the simulated population.

That is, frequency-dependent transmission:

$$ \lambda (t) \propto \frac{I(t)}{N(t)} $$

where $\lambda (t)$ represents the force of infection,
$I(t)$ the number of infectious persons in the population,
$N(t)$ the total population size
and $t$ time.

## How is density-dependent tranmission different?
The conceptual difference is that the force of infection
is proportional to the number of infectious persons rather than the
prevalence of infectious persons.
That is, for density-dependent transmission:

$$ \lambda (t) \propto I(t) $$

For either of these assumptions, 
there is also a model input parameter that will be multiplied through
by the size of the infectious population (number or prevalence).
The symbol $\beta$ is often used to represent the quantity that scales $I(t)$,
and so is equivalent to our `contact_rate` parameter for density-dependent transmission.
For frequency-dependent transmission, 
$\beta$ can again represent the constant of proportionality between
the force of infection ($\lambda(t)$) and the number of infectives ($I(t)$).
In this case, it incorporates the division by the population size ($N(t)$),
and so differs from our `contact_rate` parameter.
Unfortunately there are no consistent rules for what $\beta$ should represent,
and so it is important to examine how it is implemented in any given model.
Of course, the name of the parameter is less important than what it represents, 
which is discussed below.
Once we have calculated the force of infection, we can think of it in a similar way to 
how we think of a parameter to a standard inter-compartmental transition flow
(the difference being that the rate is dependent on quantity that emerges from the model).
As for any other such flow, it is next multiplied by the source compartment
(`susceptible`) when we come to calculating the actual number of people being infected.

Next, let's run a model with density-dependent transmission instead of frequency-dependent.

In [ ]:
dens_parameters = {
    "recovery": 0.333,
    "contact_rate": 1.,
}
sir_dens_model = get_sir_base_structure(model_config)
sir_dens_model.add_infection_density_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
sir_dens_model.run(parameters=dens_parameters)
sir_dens_values = sir_dens_model.get_outputs_df()
sir_dens_values.plot(
    labels={"index": "time", "value": "proportion"},
)

The model outputs are identical, 
which is unsurprising because this is a really trivial example.
Because the population size is fixed at a value of one,
the division by $N(t)$ has no effect on the force of infection.
However, next let's consider what happens if we scale the population size back up to 1000.
We can easily recover the exact same dynamics as we saw previously for both the
frequency and density-dependent models by dividing `population`, `seed` and `contact_rate`
by the new, larger population size.
However, we need to adjust the model's parameters too.

In [ ]:
# Scale all population-related quantities up by a factor of 1000
model_config.update(
    {
        "population": 1000.,
        "seed": 10.,
    }
)

# Scale the contact rate parameter down by a factor of 1000
dens_parameters.update(
    {
        "contact_rate": 0.001,
    }
)
large_pop_dens_model = get_sir_base_structure(model_config)
large_pop_dens_model.add_infection_density_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
large_pop_dens_model.run(parameters=dens_parameters)
large_pop_dens_values = large_pop_dens_model.get_outputs_df()
large_pop_dens_values.plot(
    labels={"index": "time", "value": "compartment size"},
)

So here we have shown that if the population size is fixed over time,
we can easily recover the same dynamics for both transmission types.
The only difference is the interpretation of the `contact_rate` parameter
that we used to produce these two simulations.
Under the assumption of frequency dependence,
we can interpret this parameter as the rate at which two specific individuals
come into effective contact in the population
(recalling that effective contact is a contact which would result in transmission
if it were to occur between a susceptible and an infectious person).
By contrast, under the assumption of density dependence,
the `contact_rate` parameter should be interpreted as the number of effective contacts
made by an infectious person per unit time.
Therefore, the parameter needs to be smaller by a factor of the size of 
the population under density dependence compared to frequency dependence.

## Changing population size
So far, this is all pretty trivial.
We have demonstrated identical dynamics under these assumptions.
Next, let's consider what would happen if the population size changes over time.
To do this, let's add deaths to all of the compartments so that the population
changes dramatically during the simulation time frame.
This unrealistic if we're thinking of the time unit as days,
but is an easy way to make sure the population size changes rapidly enough to affect things.

### Frequency dependence, declining population

In [ ]:
sir_freq_deaths_model = get_sir_base_structure(model_config)
sir_freq_deaths_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
freq_parameters.update(
    {
        "crude_death_rate": 0.05,
    }
)
sir_freq_deaths_model.add_universal_death_flows(
    "non_infection_deaths",
    Parameter("crude_death_rate"),
)
sir_freq_deaths_model.run(parameters=freq_parameters)
sir_freq_deaths_values = sir_freq_deaths_model.get_outputs_df()
sir_freq_deaths_values.plot(
    labels={"index": "time", "value": "compartment size"},
)

### Frequency dependence, unchanged proportional epidemic dynamics
It looks like the model dynamics are different, and in a sense they are.
However, if we look at the proportional sizes of the compartments,
we see that actually the dynamics are essentially the same.

In [ ]:
sir_freq_deaths_props = sir_freq_deaths_values.div(sir_freq_deaths_values.sum(axis=1), axis=0)
sir_freq_deaths_props.plot(
    labels={"index": "time", "value": "proportion"},
)

### Density dependence, declining population
Let's work through the same process for the density-dependent transmission model.
Again, if we look at the compartment sizes directly,
we can see that the model dynamics have changed.

In [ ]:
sir_dens_deaths_model = get_sir_base_structure(model_config)
sir_dens_deaths_model.add_infection_density_flow(
    name="infection", 
    contact_rate=Parameter("contact_rate"),
    source="susceptible", 
    dest="infectious",
)
dens_parameters.update(
    {
        "crude_death_rate": 0.05,
    }
)
sir_dens_deaths_model.add_universal_death_flows(
    "non_infection_deaths",
    Parameter("crude_death_rate"),
)
sir_dens_deaths_model.run(parameters=dens_parameters)
sir_dens_deaths_values = sir_dens_deaths_model.get_outputs_df()
sir_dens_deaths_values.plot(
    labels={"index": "time", "value": "compartment size"},
)

### Density dependence, changed infection dynamics with changing population size
However, perhaps more importantly we can also see that the
dynamics are different if we look at the compartment proportions over time.
Specifically, because the population shrinks as the epidemic proceeds,
the rate of infection falls as the `susceptible` and `infectious` compartments
of the model drop, which is not offset by the shrinking denominator
of the population size (as it would be under frequency dependence).
This leads to a smaller epidemic final size relative to population,
and so a greater proportion of the population remaining susceptible throughout.

In [ ]:
sir_dens_deaths_props = sir_dens_deaths_values.div(sir_dens_deaths_values.sum(axis=1), axis=0)
sir_dens_deaths_props.plot(
    labels={"index": "time", "value": "proportion"},
)